In [27]:
import pandas as pd
import duckdb
import os
from datetime import datetime, timedelta
from tqdm import tqdm

In [ ]:
con = duckdb.connect(database='dataset/JAAR_DB.duckdb', read_only=True)

Exploring the database

In [ ]:
fetch_info = """
SELECT *
FROM information_schema.tables
"""

In [ ]:
result = con.execute(fetch_info).fetchall()
print(result)

In [ ]:
fetch_info = """
SELECT *
FROM JAAR_raw
ORDER BY ChamberID, TIMESTAMP
"""

In [ ]:
result = con.execute(fetch_info).df()
print(result)

In [ ]:
print(result.info())

In [ ]:
result.to_csv('JAAR_raw.csv', index=False)

In [ ]:
fetch_info = """
SELECT ChamberID, COUNT(*) as count
FROM JAAR_raw
GROUP BY ChamberID
ORDER BY ChamberID
"""

In [ ]:
result = con.execute(fetch_info).df()
print(result)

Extracting important timestamps

In [4]:
fetch_info = """
WITH active_sessions AS (
    SELECT
        ChamberID,
        TIMESTAMP,
        ChamberStatus,
        LAG(ChamberStatus) OVER (PARTITION BY ChamberID ORDER BY TIMESTAMP) AS prev_status,
        LEAD(ChamberStatus) OVER (PARTITION BY ChamberID ORDER BY TIMESTAMP) AS next_status
    FROM JAAR_raw
    ORDER BY ChamberID, TIMESTAMP
),
session_start AS (
    SELECT
        ChamberID,
        TIMESTAMP AS start_time,
    FROM active_sessions
    WHERE ChamberStatus != 0 AND (prev_status = 0 OR prev_status IS NULL)
),
session_end AS (
    SELECT
        ChamberID,
        TIMESTAMP AS end_time,
    FROM active_sessions
    WHERE ChamberStatus != 0 AND (next_status = 0 OR next_status IS NULL)
)
SELECT
    s.ChamberID,
    s.start_time,
    e.end_time,
    DATEDIFF('second', CAST(s.start_time AS TIMESTAMP), CAST(e.end_time AS TIMESTAMP)) AS duration_seconds
FROM session_start s
JOIN LATERAL (
    SELECT e.end_time
    FROM session_end e
    WHERE e.ChamberID = s.ChamberID AND e.end_time > s.start_time
    ORDER BY e.end_time
    LIMIT 1
) e ON true
ORDER BY s.ChamberID, s.start_time
"""

In [5]:
all_active_sessions = con.execute(fetch_info).df()
print(all_active_sessions)

       ChamberID           start_time             end_time  duration_seconds
0            1.0  2025-03-04 00:34:47  2025-03-04 00:39:55               308
1            1.0  2025-03-04 01:34:47  2025-03-04 01:39:55               308
2            1.0  2025-03-04 02:34:47  2025-03-04 02:39:55               308
3            1.0  2025-03-04 03:34:47  2025-03-04 03:39:55               308
4            1.0  2025-03-04 04:34:47  2025-03-04 04:39:55               308
...          ...                  ...                  ...               ...
24189        9.0  2025-06-23 19:32:43  2025-06-23 19:37:51               308
24190        9.0  2025-06-23 20:32:43  2025-06-23 20:37:51               308
24191        9.0  2025-06-23 21:32:43  2025-06-23 21:37:51               308
24192        9.0  2025-06-23 22:32:43  2025-06-23 22:37:51               308
24193        9.0  2025-06-23 23:32:43  2025-06-23 23:37:51               308

[24194 rows x 4 columns]


In [21]:
all_active_sessions['ChamberID'].value_counts()

ChamberID
1.0    2689
6.0    2689
2.0    2688
4.0    2688
3.0    2688
5.0    2688
7.0    2688
8.0    2688
9.0    2688
Name: count, dtype: int64

In [ ]:
len(all_active_sessions)

Extracting the actual data based on the timestamps

In [6]:
result_dir = 'result'

In [ ]:
session_count = 0
idx = 0
while(idx < len(all_active_sessions)):
    session_count += 1
    
    row_1 = all_active_sessions.iloc[idx]
    row_2 = all_active_sessions.iloc[idx + 1] if idx + 1 < len(all_active_sessions) else None

    #Extract data from the first row
    chamber_id = row_1['ChamberID']
    start_time = row_1['start_time']
    end_time = row_1['end_time']
    duration_seconds = row_1['duration_seconds']
    
    start_time_dt = datetime.fromisoformat(str(start_time))
    end_time_dt = datetime.fromisoformat(str(end_time))

    buffer_start = start_time_dt - timedelta(seconds=5)
    buffer_end = end_time_dt + (timedelta(seconds=5) if row_2 is None else timedelta(seconds=0))

    print(f"Processing ChamberID {chamber_id}, session {session_count}")
    print(f"Period: {str(start_time)} to {str(end_time)} ({duration_seconds:.0f}seconds)")

    data_query = f"""
    SELECT
        TIMESTAMP,
        CH4,
        ChamberStatus,
        ChamberID,
        ChamberTC,
    FROM JAAR_raw
    WHERE ChamberID = '{chamber_id}'
    AND TIMESTAMP >= '{buffer_start}'
    AND TIMESTAMP <= '{buffer_end}'
    ORDER BY TIMESTAMP
    """

    df = con.execute(data_query).df()

    idx += 1
    
    #Extrac data from the second row if it exists
    if row_2 is not None:
        chamber_id_next = row_2['ChamberID']
        
        if chamber_id_next != chamber_id:
            continue

        start_time_next = row_2['start_time']
        end_time_next = row_2['end_time']
        duration_seconds_next = row_2['duration_seconds']

        start_time_next_dt = datetime.fromisoformat(str(start_time_next))
        end_time_next_dt = datetime.fromisoformat(str(end_time_next))

        buffer_start_next = start_time_next_dt - timedelta(seconds=0)
        buffer_end_next = end_time_next_dt + timedelta(seconds=5)

        print(f"Merging with next session period: {str(buffer_start_next)} to {str(buffer_end_next)} ({duration_seconds_next:.0f}seconds)")

        data_query = f"""
        SELECT
            TIMESTAMP,
            CH4,
            ChamberStatus,
            ChamberID,
            ChamberTC,
        FROM JAAR_raw
        WHERE ChamberID = '{chamber_id_next}'
        AND TIMESTAMP >= '{buffer_start_next}'
        AND TIMESTAMP <= '{buffer_end_next}'
        ORDER BY TIMESTAMP
        """

        df_next = con.execute(data_query).df()

        #Merge the two dataframes
        df = pd.concat([df, df_next], ignore_index=True)

        idx += 1
    
    #Save the dataframe to a CSV file
    if len(df) > 0:
        session_key = f"Chamber_{int(chamber_id)}_Session_{session_count}"
        csv_filename = os.path.join(result_dir, f"{session_key}.csv")
        df.to_csv(csv_filename, index=False)
    else:
        print(f"No data found for ChamberID {chamber_id} in session {session_count}. Skipping...")

print(f"Total sessions processed: {session_count}")

In [ ]:
print(df)

In [ ]:
session_count = 0
segment_count = 0
idx = 0
time_sample = 400
dfs = []
_len = len(all_active_sessions)
pbar = tqdm(total=_len, desc="Processing session")
total_duration = 0
while(True):
    session_count += 1
    row = all_active_sessions.iloc[idx]
    next_row = all_active_sessions.iloc[idx+1] if idx < _len-1 else None

    #Extract data from the first row
    chamber_id = row['ChamberID']
    start_time = row['start_time']
    end_time = row['end_time']
    duration_seconds = row['duration_seconds']
    
    start_time_dt = datetime.fromisoformat(str(start_time))
    end_time_dt = datetime.fromisoformat(str(end_time))

    #print(f"Processing ChamberID {chamber_id}, session {session_count}")
    #print(f"Period: {str(start_time)} to {str(end_time)} ({duration_seconds:.0f}seconds)")

    data_query = f"""
    SELECT
        TIMESTAMP,
        CH4,
        ChamberStatus,
        ChamberID,
        ChamberTC,
    FROM JAAR_raw
    WHERE ChamberID = '{chamber_id}'
    AND TIMESTAMP >= '{start_time_dt}'
    AND TIMESTAMP <= '{end_time_dt}'
    ORDER BY TIMESTAMP
    """

    df = con.execute(data_query).df()

    # Update some parameter
    idx += 1
    total_duration += duration_seconds

    # Check if the recent queried df exceed the time sample, if yes split it then append else just append
    if (total_duration >= time_sample):
        time_diff = total_duration - time_sample + 1
        idx_sep = duration_seconds - time_diff
        df_1, df_2 = df.iloc[:idx_sep], df.iloc[idx_sep:]
        dfs.append(df_1)
        dfs.append(df_2)
    else:
        dfs.append(df)

    # Check whether the total duration has exceed the time sample or not
    # Check whether the index has exceed all active sessions
    all_session_processed = (idx >= _len)
    different_id = (row['ChamberID'] != next_row['ChamberID']) if next_row is not None else True
    time_exceed = (total_duration > time_sample)
    time_equal = (total_duration == time_sample)
    if(time_exceed or time_equal or different_id or all_session_processed):
        segment_count += 1
        if time_exceed and not different_id:
            # Combine until dfs[:-1]
            combined_df = pd.concat(dfs[:-1], ignore_index=True)

            # Leave the last element in the dfs
            dfs[:] = [dfs[-1]]

            # Reset the total duration to be the residue time
            total_duration = time_diff
        else:
            # Combine all dfs
            combined_df = pd.concat(dfs, ignore_index=True)

            # Reset the list
            dfs[:] = []

            # Reset the total duration to be 0
            total_duration = 0

        #Save the dataframe to a CSV file
        if len(combined_df) > 0:
            session_key = f"Chamber_{int(chamber_id)}_Segment_{segment_count}"
            csv_filename = os.path.join(result_dir, f"{session_key}.csv")
            combined_df.to_csv(csv_filename, index=False)
        else:
            print(f"No data found for ChamberID {chamber_id} in session {session_count}. Skipping...")
        
        # Break if all session has been processed
        if(all_session_processed):
            break

print(f"Total sessions processed: {session_count}")

Processing ChamberID 1.0, session 1
Period: 2025-03-04 00:34:47 to 2025-03-04 00:39:55 (308seconds)
Processing ChamberID 1.0, session 2
Period: 2025-03-04 01:34:47 to 2025-03-04 01:39:55 (308seconds)
Processing ChamberID 1.0, session 3
Period: 2025-03-04 02:34:47 to 2025-03-04 02:39:55 (308seconds)
Processing ChamberID 1.0, session 4
Period: 2025-03-04 03:34:47 to 2025-03-04 03:39:55 (308seconds)
Processing ChamberID 1.0, session 5
Period: 2025-03-04 04:34:47 to 2025-03-04 04:39:55 (308seconds)
Processing ChamberID 1.0, session 6
Period: 2025-03-04 05:34:47 to 2025-03-04 05:39:55 (308seconds)
Processing ChamberID 1.0, session 7
Period: 2025-03-04 06:34:47 to 2025-03-04 06:39:55 (308seconds)
Processing ChamberID 1.0, session 8
Period: 2025-03-04 07:34:47 to 2025-03-04 07:39:55 (308seconds)
Processing ChamberID 1.0, session 9
Period: 2025-03-04 08:34:47 to 2025-03-04 08:39:55 (308seconds)
Processing ChamberID 1.0, session 10
Period: 2025-03-04 09:34:47 to 2025-03-04 09:39:55 (308seconds)